In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch

prompt = lambda question: f"""\
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant for SUNY Brockport, a public college in Brockport, New York. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

{question} [/INST]
"""

REPO_ID = "msaad02/BrockportGPT-7b"

generator = pipeline(
    task='text-generation', 
    tokenizer=AutoTokenizer.from_pretrained(REPO_ID),
    model=AutoModelForCausalLM.from_pretrained(
        pretrained_model_name_or_path=REPO_ID,
        quantization_config=BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type='nf4'
        ),    
    ), 
    torch_dtype=torch.bfloat16,
    device_map={"": 0},
)

In [14]:
question="How can I apply to SUNY Brockport?"

output=generator(prompt(question), do_sample=True, temperature=0.7, max_new_tokens=512)
output[0]["generated_text"].split("[/INST]\n")[1]

"Applying to SUNY Brockport is a straightforward process. You can start by selecting the 'Apply Now' button on our homepage, which will guide you through the application process. Our Admissions Team is always available to assist you with any questions or concerns you may have. We can't wait to welcome you to our community of Golden Eagles!"